# Chainerで多層パーセプトロンを作ろう
## MNISTデータで手書き文字認識

このファイルは以下のGitHubディレクトリにあるtrain_mnist.pyをipython notebookにペタペタ貼ったもの。
https://github.com/pfnet/chainer/tree/master/examples/mnist

- [Chainer@GitHub](https://github.com/pfnet/chainer.git)

このipynbファイルを、chainer

In [48]:
%matplotlib inline
import matplotlib.pyplot as plt

In [49]:
import numpy as np

In [50]:
from chainer import Variable, Chain, optimizers
import chainer.functions as F
import chainer.links as L

## データの読み込み

In [51]:
import data

In [52]:
mnist = data.load_mnist_data()

In [53]:
mnist

{'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([5, 0, 4, ..., 4, 5, 6], dtype=uint8)}

In [54]:
mnist['data'].max()

255

In [55]:
x_all = mnist['data'].astype(np.float32) / 255
y_all = mnist['target'].astype(np.int32)

In [56]:
x_train, x_test = np.split(x_all, [60000])
y_train, y_test = np.split(y_all, [60000])

## モデルを定義

In [57]:
class MLP(Chain):
    def __init__(self):
        super(MLP, self).__init__(
            l1 = L.Linear(784, 100),
            l2 = L.Linear(100, 100),
            l3 = L.Linear(100, 10),
        )
        
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y

損失関数と分類精度の計算をするために、MLP chainの上にclassifier chainを作る。

In [58]:
class MyClassifier(Chain):
    """
    Compute accuracy and loss.
    
    Returns:
        loss
    """
    
    def __init__(self, predictor):
        super(MyClassifier, self).__init__(
            predictor=predictor
        )
    
    def __call__(self, x, t):
        y = self.predictor(x)
        self.loss = F.softmax_cross_entropy(y, t)
        self.accuracy = F.accuracy(y, t)
        return self.loss

今、定義したMyClassifierクラスと似たクラスが、chainer.links.Classifierに定義されているので、通常はそっちをつかうべし。

In [59]:
model = MyClassifier(MLP())
#model = L.Classifier(MLP())    # <- 本来ならこっちの方が良い

In [60]:
optimizer = optimizers.SGD()

In [61]:
optimizer.setup(model)

### 書き方１

In [62]:
batchsize = 100
datasize = 60000

In [63]:
for epoch in range(20):
    print('epoch %d' % epoch)
    indices = np.random.permutation(datasize)
    for i in range(0, datasize, batchsize):
        x = Variable(x_train[indices[i : i + batchsize]])
        t = Variable(y_train[indices[i : i + batchsize]])
        
        optimizer.update(model, x, t)

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19


### 書き方２

In [15]:
batchsize = 100
datasize = 60000

In [64]:
for epoch in range(20):
    print('epoch %d' % epoch)
    
    # Compute test/validation error
    sum_loss, sum_accuracy = 0, 0
    for i in range(0, 10000, batchsize):
        x = Variable(x_test[i : i + batchsize])
        t = Variable(y_test[i : i + batchsize])
        loss = model(x, t)
        sum_loss += loss.data * batchsize
        sum_accuracy += model.accuracy.data * batchsize

    mean_loss = sum_loss / 10000
    mean_accuracy = sum_accuracy / 10000

    print("mean loss: %.5f\t mean accuracy: %.5f" % (mean_loss, mean_accuracy))    
    
    indices = np.random.permutation(datasize)
    for i in range(0, datasize, batchsize):
        x = Variable(x_train[indices[i : i + batchsize]])
        t = Variable(y_train[indices[i : i + batchsize]])
        
        model.zerograds()
        loss = model(x, t)
        loss.backward()
                
        optimizer.update()

epoch 0
mean loss: 0.16905	 mean accuracy: 0.94960
epoch 1
mean loss: 0.16476	 mean accuracy: 0.95150
epoch 2
mean loss: 0.16115	 mean accuracy: 0.95200
epoch 3
mean loss: 0.15899	 mean accuracy: 0.95290
epoch 4
mean loss: 0.15248	 mean accuracy: 0.95430
epoch 5
mean loss: 0.14985	 mean accuracy: 0.95500
epoch 6
mean loss: 0.14653	 mean accuracy: 0.95650
epoch 7
mean loss: 0.14390	 mean accuracy: 0.95710
epoch 8
mean loss: 0.14173	 mean accuracy: 0.95880
epoch 9
mean loss: 0.13728	 mean accuracy: 0.95860
epoch 10
mean loss: 0.13546	 mean accuracy: 0.95980
epoch 11
mean loss: 0.13285	 mean accuracy: 0.96090
epoch 12
mean loss: 0.12969	 mean accuracy: 0.96120
epoch 13
mean loss: 0.12803	 mean accuracy: 0.96220
epoch 14
mean loss: 0.12677	 mean accuracy: 0.96290
epoch 15
mean loss: 0.12319	 mean accuracy: 0.96320
epoch 16
mean loss: 0.12123	 mean accuracy: 0.96430
epoch 17
mean loss: 0.12186	 mean accuracy: 0.96340
epoch 18
mean loss: 0.11771	 mean accuracy: 0.96430
epoch 19
mean loss: 0.